<a href="https://colab.research.google.com/github/davidelgas/DataSciencePortfolio/blob/main/Language_Models/NLP_with_LDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Topic Modeling with Latent Dirichlet Allocation (LDA)
https://nlp.stanford.edu/events/illvi2014/papers/sievert-illvi2014.pdf

# Introduction

In this notebook I will be using LDA to identify unique topics in a corpus.

# Provision Enviorment

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# Install all necessary packages
!pip install numpy==1.24.3
!pip install pandas
!pip install nltk
!pip install gensim==4.3.0
!pip install pyLDAvis
!pip install sumy
!pip install snowflake-connector-python
!pip install requests
!pip install beautifulsoup4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.3/60.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 57.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 97.5 MB/s eta 0:00:00
  Created wheel for fst-pso: filename=fst_pso-1.8.1-py3-none-any.whl size=20430 sha256=1f4e8283d2d570dfc9be526e51b452f6c6a566e2278b35cc911d2d9144006840
  Stored in directory: /root/.cache/pip/wheels/69/f5/e5/18ad53fe1ed6b2af9fad05ec052e4acbac8e92441df44bad2e
  Created wheel for miniful: filename=miniful-0.0.6-py3-none-any.whl size=3507 sha256=9956ffa4b0e8dc143ae05da2c3079add97c791ca1854d9e5d6203ed324468666
  Stored in directory: /root/.cach

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 6.3 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of pyfume to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 73.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.6/59.6 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
  Attempting uninstall: pyfume
    Found existing installation: pyFUME 0.3.4
    Uninstalling pyFUME-0.3.4:
      Successfully uninstalled pyFUME-0.3.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==2.2.2, but you have pandas 2

In [5]:
# Import all necessary libraries
import os
import numpy as np
import pandas as pd
import nltk
import random
import re
import requests
from bs4 import BeautifulSoup
import snowflake.connector

# Download NLTK resources
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')

# NLTK imports
from nltk.tokenize import RegexpTokenizer, sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet, stopwords

# Gensim imports
from gensim import corpora
from gensim.corpora import Dictionary
from gensim.models import LdaModel
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

# PyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

# Sumy libraries
from sumy.parsers.plaintext import PlaintextParser
from sumy.summarizers.lex_rank import LexRankSummarizer
from sumy.nlp.tokenizers import Tokenizer

print("All imports successful!")

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


All imports successful!


# Corpus

The corpus used was assembled using Beautiful Soup to scrape a pubic forum specific to the BMW E9 (www.e9coupe.com). The data was compiled and stored in a Snowflake database for multiple NLP projects, including LDA, RAG, GRU and LSTM. Furture ideas include supplementing the forum text with an existing users guide specific to this model.

In [6]:
path_to_credentials = '/content/drive/Othercomputers/My Mac/Git/credentials/snowflake_credentials.txt'
lda_model_path = '/content/drive/Othercomputers/My Mac/Git/Language_Models/lda_model'
dictionary_path = '/content/drive/Othercomputers/My Mac/Git/Language_Models/datasets'



## Load data

In [7]:
# Data is stored in Snowflake

# Main sequence

def load_credentials(path_to_credentials):
    with open(path_to_credentials, 'r') as file:
        for line_num, line in enumerate(file, start=1):
            line = line.strip()
            if line and '=' in line:
                key, value = line.split('=')
                os.environ[key] = value
            else:
                print(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")

def fetch_data_from_snowflake():
    conn = snowflake.connector.connect(
        user=os.environ.get('USER'),
        password=os.environ.get('PASSWORD'),
        account=os.environ.get('ACCOUNT'),
    )

    cur = conn.cursor()

    query = """
    SELECT THREAD_TITLE, THREAD_FIRST_POST FROM "E9_CORPUS"."E9_CORPUS_SCHEMA"."E9_FORUM_CORPUS";
    """
    cur.execute(query)
    e9_forum_corpus = cur.fetch_pandas_all()

    cur.close()
    conn.close()

    return e9_forum_corpus




# Load credentials
# load_credentials(path_to_credentials)

# Fetch data from Snowflake
# df_raw = fetch_data_from_snowflake()

# df_tensorboard = df_raw.copy()

# df_tensorboard.to_csv('/content/drive/MyDrive/Colab Notebooks/NLP/NLP_with_Tensorboard/df_tensorboard.csv', index=False)


In [8]:
#Process data for LDA

lda_num_topics = 10

def load_credentials(path_to_credentials):
    with open(path_to_credentials, 'r') as file:
        for line_num, line in enumerate(file, start=1):
            line = line.strip()
            if line and '=' in line:
                key, value = line.split('=')
                os.environ[key] = value
            else:
                print(f"Issue with line {line_num} in {path_to_credentials}: '{line}'")


def fetch_data_from_snowflake():
    conn = snowflake.connector.connect(
        user=os.environ.get('USER'),
        password=os.environ.get('PASSWORD'),
        account=os.environ.get('ACCOUNT'),
    )

    cur = conn.cursor()

    query = """
    SELECT THREAD_TITLE, THREAD_FIRST_POST FROM "E9_CORPUS"."E9_CORPUS_SCHEMA"."E9_FORUM_CORPUS";
    """
    cur.execute(query)
    e9_forum_corpus = cur.fetch_pandas_all()

    cur.close()
    conn.close()

    return e9_forum_corpus

# Create a longer form "TITLE" for potential use as a QUESTION
def engineer_data(e9_forum_corpus):
    e9_forum_corpus['THREAD_TITLE_EXP'] = e9_forum_corpus['THREAD_TITLE'] + " " + e9_forum_corpus['THREAD_FIRST_POST']
    return e9_forum_corpus

# Drop NA and Stopwords and Lemmatize
def preprocess_data(df):
    #df = df[['THREAD_TITLE_EXP']].copy()
    df = df[['THREAD_TITLE']].copy()
    df.dropna(inplace=True)
    df['THREAD_TITLE'] = df['THREAD_TITLE'].astype(str)

    #df['THREAD_TITLE_EXP'] = df['THREAD_TITLE_EXP'].astype(str)

    lemmatizer = WordNetLemmatizer()
    stop_words = set(stopwords.words('english')).union({'car', 'csi', 'cs', 'csl', 'e9', 'coupe', 'http', 'https', 'www', 'ebay', 'bmw', 'html'})

    # Remove URLs
    def remove_urls(text):
        url_pattern = re.compile(r'https?://\S+|www\.\S+')
        return url_pattern.sub(r'', text)

    # Function to preprocess text
    def preprocess(text):
        text = remove_urls(text)
        return [lemmatizer.lemmatize(word) for word in text.lower().split() if word not in stop_words]

    #df['PROCESSED'] = df['THREAD_TITLE_EXP'].map(preprocess)
    df['PROCESSED'] = df['THREAD_TITLE'].map(preprocess)

    return df

def vectorize_data(df):
    dictionary = Dictionary(df['PROCESSED'])
    corpus = [dictionary.doc2bow(doc) for doc in df['PROCESSED']]
    return df, dictionary, corpus

def train_lda_model(corpus, dictionary, num_topics=lda_num_topics, random_state=42, passes=10):
    lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=num_topics, random_state=random_state, passes=passes)
    return lda

def review_topics(lda):
    for idx, topic in lda.print_topics(-1):
        print(f"Topic: {idx} \nWords: {topic}\n")

def assign_topics(lda, corpus, df):
    topics = [lda[doc] for doc in corpus]
    df['TOPICS'] = [[(int(topic[0]), float(topic[1])) for topic in doc] for doc in topics]  # Ensure topics are JSON serializable
    return df

def prepare_visualization_data(lda, corpus, dictionary):
    vis_data = gensimvis.prepare(lda, corpus, dictionary)
    return vis_data



In [9]:
# Generate LDA

# New UDF to save the LDA model and dictionary
def save_lda_model_and_dictionary(lda, dictionary, lda_model_path, dictionary_path):
    # Create directories if they don't exist
    import os
    os.makedirs(os.path.dirname(lda_model_path), exist_ok=True)
    os.makedirs(dictionary_path, exist_ok=True)

    # Save model - append a filename if lda_model_path is a directory
    if os.path.isdir(lda_model_path):
        lda_model_path = os.path.join(lda_model_path, 'lda_model')
    lda.save(lda_model_path)

    # Save dictionary - create a specific filename in the dictionary_path
    dictionary_file_path = os.path.join(dictionary_path, 'dictionary.dict')
    dictionary.save(dictionary_file_path)

    print(f"LDA model saved to: {lda_model_path}")
    print(f"Dictionary saved to: {dictionary_file_path}")

# Load credentials
load_credentials(path_to_credentials)

# Fetch data from Snowflake
e9_forum_corpus = fetch_data_from_snowflake()

# Engineer the data
e9_forum_corpus = engineer_data(e9_forum_corpus)

# Preprocess the data
df = preprocess_data(e9_forum_corpus)

# Vectorize the data
df, dictionary, corpus = vectorize_data(df)

# Train the LDA Model
lda = train_lda_model(corpus, dictionary)

# Save the LDA model and dictionary
save_lda_model_and_dictionary(lda, dictionary, lda_model_path, dictionary_path)

# Review the Topics
review_topics(lda)

# Function to assign topics to documents
def assign_topics(lda, corpus, df):
    topic_assignments = []
    for i, row in enumerate(lda[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        topic_assignments.append(row[0][0])
    df['Topic'] = topic_assignments
    return df

# Assign Documents to Topics
df = assign_topics(lda, corpus, df)

# Prepare the visualization data
vis_data = prepare_visualization_data(lda, corpus, dictionary)

# Visualize
pyLDAvis.display(vis_data)


LDA model saved to: /content/drive/Othercomputers/My Mac/Git/Language_Models/lda_model
Dictionary saved to: /content/drive/Othercomputers/My Mac/Git/Language_Models/datasets/dictionary.dict
Topic: 0 
Words: 0.051*"seat" + 0.026*"link" + 0.026*"looking" + 0.026*"company" + 0.026*"posted" + 0.026*"italian" + 0.026*"making" + 0.026*"3.5" + 0.026*"brand" + 0.026*"liter"

Topic: 1 
Words: 0.048*"sale" + 0.048*"wheel" + 0.048*"steering" + 0.048*"brake" + 0.025*"question" + 0.024*"link" + 0.024*")" + 0.024*"(" + 0.024*"germany" + 0.024*"swap"

Topic: 2 
Words: 0.069*"engine" + 0.047*"close" + 0.047*"ratio" + 0.046*"option" + 0.024*"opinion" + 0.024*"gear" + 0.024*"overdrive" + 0.024*"wanted:" + 0.024*"box?" + 0.024*"glad"

Topic: 3 
Words: 0.041*"-" + 0.041*"back" + 0.041*"getting" + 0.021*"interior" + 0.021*"welcome" + 0.021*"noise" + 0.021*"," + 0.021*"thanks" + 0.021*"more?" + 0.021*"want"

Topic: 4 
Words: 0.051*"picture" + 0.051*"lsd" + 0.051*"bucket" + 0.051*"dear" + 0.051*"old" + 0.026

In [11]:
#Create representative sentences for each topic


representative_sentences = []

def score_text_block(text, topic_words):
    if isinstance(text, str):
        return sum(1 for word in topic_words if word in text)
    return 0


def create_representative_sentence(lda, df, topic_id, top_n=20):
    topic_words = [word for word, prob in lda.show_topic(topic_id, topn=top_n)]
    #df['SCORE'] = df['THREAD_TITLE_EXP'].map(lambda x: score_text_block(x, topic_words))
    #representative_sentence = df.loc[df['SCORE'].idxmax(), 'THREAD_TITLE_EXP']
    df['SCORE'] = df['THREAD_TITLE'].map(lambda x: score_text_block(x, topic_words))
    representative_sentence = df.loc[df['SCORE'].idxmax(), 'THREAD_TITLE']

    return representative_sentence



for topic_id in range(lda_num_topics):
    try:
        sentence = create_representative_sentence(lda, e9_forum_corpus, topic_id)
        representative_sentences.append({'Topic': topic_id, 'Representative Sentence': sentence})
        print(f"Topic {topic_id}: {sentence}")
    except IndexError as e:
        print(f"Error with topic {topic_id}: {e}")

# Save to CSV
output_df = pd.DataFrame(representative_sentences)
output_df.to_csv('/content/drive/MyDrive/Colab Notebooks/datasets/e9/representative_sentences.csv', index=False)


Topic 0: Looking for link I posted about italian company making seats
Topic 1: Further steering wheel question ( link )
Topic 2: Opinions wanted: Close ratio or overdrive gear box?
Topic 3: DOB - Anybody want more?
Topic 4: Does anyone know of a 3.64 lsd for sale?
Topic 5: looking for radio console
Topic 6: CSL project update
Topic 7: Need tool box for a 3200 CS 1964
Topic 8: New 6 series type Airdam Available Now.
Topic 9: Slighty off topic, Pictures for Mike and Tony, Next inport?
